# 第1章: 基本的なプロンプト構造

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## レッスン

Anthropicは、従来の[Text Completions API](https://docs.anthropic.com/claude/reference/complete_post)と現在の[Messages API](https://docs.anthropic.com/claude/reference/messages_post)の2つのAPIを提供しています。このチュートリアルでは、Messages APIのみを使用します。

Messages APIを使用してClaudeを呼び出すには、最低限以下のパラメータが必要です：
- `model`：呼び出したいモデルの[APIモデル名](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations)

- `max_tokens`：停止するまでに生成する最大トークン数。Claudeはこの最大値に達する前に停止する場合があることに注意してください。このパラメータは生成する絶対最大トークン数のみを指定します。さらに、これは*ハード*ストップです。つまり、単語や文の途中でClaudeが生成を停止する可能性があります。

- `messages`：入力メッセージの配列。私たちのモデルは、`user`と`assistant`の会話ターンの交代で動作するよう訓練されています。新しい`Message`を作成する際、messagesパラメータで以前の会話ターンを指定し、モデルが会話の次の`Message`を生成します。
  - 各入力メッセージは、`role`と`content`を持つオブジェクトである必要があります。単一の`user`ロールメッセージを指定することも、複数の`user`と`assistant`メッセージを含めることもできます（その場合は交互になる必要があります）。最初のメッセージは常にuserロールを使用する必要があります。

オプションパラメータもあります：
- `system`：システムプロンプト - 詳細は以下で説明します。
  
- `temperature`：Claudeの応答の変動性の度合い。これらのレッスンと演習では、`temperature`を0に設定しています。

すべてのAPIパラメータの完全なリストについては、[APIドキュメント](https://docs.anthropic.com/claude/reference/messages_post)をご覧ください。

### 例

Claudeが正しくフォーマットされたプロンプトにどのように応答するかを見てみましょう。以下の各セルについて、セルを実行し（`shift+enter`）、Claudeの応答がブロックの下に表示されます。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

: 

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

: 

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

次に、正しいMessages APIフォーマットを含まないプロンプトを見てみましょう。これらの不正なフォーマットのプロンプトについて、Messages APIはエラーを返します。

まず、`messages`配列に`role`と`content`フィールドが不足しているMessages API呼び出しの例があります。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

これは、`user`と`assistant`ロールの交代に失敗したプロンプトです。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

`user`と`assistant`メッセージは**交互でなければならず**、メッセージは**`user`ターンで始まらなければなりません**。プロンプト内に複数の`user`と`assistant`のペアを持つことができます（複数ターンの会話をシミュレートするように）。また、最終的な`assistant`メッセージに単語を入れて、Claudeに中断したところから続けさせることもできます（詳細は後の章で説明します）。

#### システムプロンプト

**システムプロンプト**も使用できます。システムプロンプトは、「ユーザー」ターンで質問やタスクを提示する前に、**Claudeにコンテキスト、指示、ガイドラインを提供する**方法です。

構造的に、システムプロンプトは`user`と`assistant`メッセージのリストとは別に存在し、したがって別の`system`パラメータに属します（ノートブックの[セットアップ](#setup)セクションの`get_completion`ヘルパー関数の構造を見てください）。

このチュートリアル内で、システムプロンプトを利用する可能性がある場合は、完了関数に`system`フィールドを提供しています。システムプロンプトを使用したくない場合は、単に`SYSTEM_PROMPT`変数を空の文字列に設定してください。

#### システムプロンプトの例

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

なぜシステムプロンプトを使うのでしょうか？**よく書かれたシステムプロンプトは、Claudeのパフォーマンスをさまざまな方法で向上させることができます**。例えば、ルールや指示に従うClaudeの能力を向上させることができます。詳細については、Claudeで[システムプロンプトを使用する方法](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)に関するドキュメントをご覧ください。

これから演習に入ります。上記の内容を変更することなくレッスンプロンプトを実験したい場合は、レッスンノートブックの一番下までスクロールして[**サンプル実験場**](#example-playground)をご覧ください。

---

## 演習
- [演習 1.1 - 3まで数える](#exercise-11---counting-to-three)
- [演習 1.2 - システムプロンプト](#exercise-12---system-prompt)

### 演習 1.1 - 3まで数える
適切な`user` / `assistant`フォーマットを使用して、以下の`PROMPT`を編集してClaudeに**3まで数えさせてください**。出力は、あなたの解決策が正しいかどうかも示します。

In [ ]:
# プロンプト - これが変更すべき唯一のフィールドです
PROMPT = "[このテキストを置き換えてください]"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正解性を評価する関数
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Claudeの応答と対応する評価を印刷
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### 演習 1.2 - システムプロンプト

Claudeが3歳の子供のように応答するよう`SYSTEM_PROMPT`を修正してください。

In [ ]:
# システムプロンプト - これが変更すべき唯一のフィールドです
SYSTEM_PROMPT = "[このテキストを置き換えてください]"

# プロンプト
PROMPT = "空はどのくらい大きいですか？"

# Claudeの応答を取得
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 演習の正解性を評価する関数
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Claudeの応答と対応する評価を印刷
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### おめでとうございます！

この時点ですべての演習を解いた場合、次の章に進む準備ができました。楽しいプロンプティングを！

---

## サンプル実験場

ここは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響を与えるかを確認する場所です。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))